In [0]:
# Coding on Microsoft Azure notebook
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import FloatType
from pyspark.sql.functions import col
from pyspark.sql.functions import substring
from pyspark.sql.functions import to_date
from pyspark.sql import functions as F
from pyspark.sql.window import Window

In [0]:
# step 1.1 Load the global weather data into your big data technology of choice.
data_w = spark.read.option("header", "true").csv("/FileStore/tables/data/2019/*.gz")

In [0]:
# step 1.2 Join the stationlist.csv with the countrylist.csv to get the full country name for each station number.
# load two csv files
stationlist = spark.read.csv("/FileStore/tables/stationlist.csv", header=True, inferSchema=True)
countrylist = spark.read.csv("/FileStore/tables/countrylist.csv", header=True, inferSchema=True)
# join two csv files by country names
# I want to get full country name for stationlist, I left join stationlist with countrylist, in order to not have 2 COUNTRY_ABBR, we did a drop at the end
stationlist_fullname = stationlist.join(countrylist, stationlist.COUNTRY_ABBR == countrylist.COUNTRY_ABBR,how='left').drop(countrylist.COUNTRY_ABBR)
stationlist_fullname.show()

+------+------------+--------------+
STN_NO|COUNTRY_ABBR| COUNTRY_FULL|
+------+------------+--------------+
012240| NO| NORWAY|
020690| SW| SWEDEN|
020870| SW| SWEDEN|
021190| SW| SWEDEN|
032690| UK|UNITED KINGDOM|
033450| UK|UNITED KINGDOM|
039290| UK|UNITED KINGDOM|
039790| EI| IRELAND|
040480| IC| ICELAND|
041300| IC| ICELAND|
060100| FO| FAROE ISLANDS|
061443| DA| DENMARK|
063401| NL| NETHERLANDS|
071910| FR| FRANCE|
092640| GM| GERMANY|
123766| PL| POLAND|
125990| PL| POLAND|
129700| HU| HUNGARY|
132240| HR| CROATIA|
156500| BU| BULGARIA|
+------+------------+--------------+
only showing top 20 rows

In [0]:
# step 1.3 Join the global weather data with the full country names by station number.
data_full_Cname = data_w.join(stationlist_fullname, data_w['STN---']==stationlist_fullname['STN_NO'], how = 'left').drop(stationlist_fullname.STN_NO)
data_full_Cname.show()

+------+-----+--------+----+----+------+-----+-----+----+-----+-----+-----+-----+-----+-----+------+------------+------------+
STN---| WBAN|YEARMODA|TEMP|DEWP| SLP| STP|VISIB|WDSP|MXSPD| GUST| MAX| MIN| PRCP| SNDP|FRSHTT|COUNTRY_ABBR|COUNTRY_FULL|
+------+-----+--------+----+----+------+-----+-----+----+-----+-----+-----+-----+-----+-----+------+------------+------------+
066720|99999|20190101|40.6|36.2|1033.1|979.1| 10.4| 3.0| 9.7| 18.5|42.8*|34.9*|0.01G|999.9|000000| SZ| SWITZERLAND|
066720|99999|20190102|37.6|26.6|1033.7|979.4| 5.1|13.3| 20.6| 35.7|42.3*|33.3*|0.10G|999.9|011000| SZ| SWITZERLAND|
066720|99999|20190103|29.6|19.0|1039.1|983.6| 13.7| 3.2| 8.0| 30.7|33.8*| 22.5|0.13G|999.9|001000| SZ| SWITZERLAND|
066720|99999|20190104|29.1|20.1|1038.2|982.7| 13.6| 2.4| 4.3|999.9|33.3*| 23.2|0.39G|999.9|000000| SZ| SWITZERLAND|
066720|99999|20190105|31.6|28.8|1035.2|980.1| 4.4| 1.5| 3.5|999.9|35.1*| 28.8|0.00G|999.9|011000| SZ| SWITZERLAND|
066720|99999|20190106|36.5|33.4|1033.9|979.4| 4.0| 2.9| 6.4| 12.6|37.8*| 31.5|0.41G|999.9|010000| SZ| SWITZERLAND|
066720|99999|20190107|36.2|32.1|1034.3|979.8| 8.9| 2.2| 4.7| 9.9|39.6*|33.3*|0.03G| 0.8|010000| SZ| SWITZERLAND|
066720|99999|20190108|36.7|31.6|1024.1|970.2| 6.0| 7.6| 22.3| 38.5|42.3*|32.0*|0.04G|999.9|010000| SZ| SWITZERLAND|
066720|99999|20190109|35.1|28.6|1016.6|962.9| 2.9|14.1| 22.0| 42.3|39.2*|31.5*|0.76G| 0.4|011000| SZ| SWITZERLAND|
066720|99999|20190110|31.8|26.5|1023.2|968.8| 8.5| 3.8| 9.9| 26.4|34.9*| 29.5|0.29G|999.9|001000| SZ| SWITZERLAND|
066720|99999|20190111|29.9|23.7|1028.2|973.4| 9.0| 1.6| 3.7|999.9|33.1*| 25.3|0.05G|999.9|011000| SZ| SWITZERLAND|
066720|99999|20190112|32.4|30.1|1027.6|973.0| 6.1| 1.4| 3.5|999.9|35.1*| 29.1|0.05G|999.9|011000| SZ| SWITZERLAND|
066720|99999|20190113|34.1|33.1|1017.1|963.3| 7.1| 2.8| 7.0| 12.4|36.5*| 31.8|0.65G| 0.8|011000| SZ| SWITZERLAND|
066720|99999|20190114|42.1|32.7|1014.1|961.3| 3.3|19.4| 24.5| 54.4|47.3*| 34.5|1.50G| 0.4|011000| SZ| SWITZERLAND|
066720|99999|20190115|36.9|29.4|1024.6|970.7| 12.9| 4.6| 17.1| 41.4|42.8*|28.4*|0.56G|999.9|010000| SZ| SWITZERLAND|
066720|99999|20190116|31.4|27.4|1018.0|963.9| 13.8| 2.5| 8.4| 16.5|39.4*| 24.8|0.00G|999.9|000000| SZ| SWITZERLAND|
066720|99999|20190117|37.4|31.8|1013.2|960.0| 10.3| 4.8| 14.2| 28.7|45.9*| 28.2|0.00G|999.9|010000| SZ| SWITZERLAND|
066720|99999|20190118|31.8|23.1|1019.2|965.1| 10.5| 4.3| 14.0| 26.6|36.5*|25.9*|0.06G|999.9|010000| SZ| SWITZERLAND|
066720|99999|20190119|27.0|20.4|1017.2|962.6| 12.4| 2.1| 3.5|999.9|33.3*| 21.7|0.00G|999.9|000000| SZ| SWITZERLAND|
066720|99999|20190120|30.3|22.9|1015.9|961.7| 14.2| 2.2| 6.6| 10.1|38.3*| 23.0|0.00G|999.9|000000| SZ| SWITZERLAND|
+------+-----+--------+----+----+------+-----+-----+----+-----+-----+-----+-----+-----+-----+------+------------+------------+
only showing top 20 rows

In [0]:
data_full_Cname = data_full_Cname.withColumn("TEMP", data_full_Cname["TEMP"].cast(FloatType())).withColumn("WDSP", data_full_Cname["WDSP"].cast(FloatType()))

# step 2.1 Which country had the hottest average mean temperature over the year?
# filter missing value for temp
data_full_Cname_rmM = data_full_Cname.where(data_full_Cname['TEMP']!=9999.9)

avg_temp = data_full_Cname_rmM.groupBy('COUNTRY_FULL').avg('TEMP').select(col('COUNTRY_FULL'),col('avg(TEMP)').alias('avg_temp'))
hot_country_1st = avg_temp.orderBy('avg_temp',ascending=False).first()['COUNTRY_FULL']
print('The country with hohtst average men temperature over the year is: %s' % hot_country_1st)

The country with hohtst average men temperature over the year is: DJIBOUTI

In [0]:
# step 2.2 Which country had the most consecutive days of tornadoes/funnel cloud formations?
tor = data_full_Cname.withColumn("tor", substring(col("FRSHTT"), -1, 1))
tor = tor.na.drop()
tor = tor.withColumn("dt", to_date(col("YEARMODA"), "yyyyMMdd"))
tor.show()

+------+-----+--------+----+----+------+-----+-----+----+-----+-----+-----+-----+-----+-----+------+------------+------------+---+----------+
STN---| WBAN|YEARMODA|TEMP|DEWP| SLP| STP|VISIB|WDSP|MXSPD| GUST| MAX| MIN| PRCP| SNDP|FRSHTT|COUNTRY_ABBR|COUNTRY_FULL|tor| dt|
+------+-----+--------+----+----+------+-----+-----+----+-----+-----+-----+-----+-----+-----+------+------------+------------+---+----------+
066720|99999|20190101|40.6|36.2|1033.1|979.1| 10.4| 3.0| 9.7| 18.5|42.8*|34.9*|0.01G|999.9|000000| SZ| SWITZERLAND| 0|2019-01-01|
066720|99999|20190102|37.6|26.6|1033.7|979.4| 5.1|13.3| 20.6| 35.7|42.3*|33.3*|0.10G|999.9|011000| SZ| SWITZERLAND| 0|2019-01-02|
066720|99999|20190103|29.6|19.0|1039.1|983.6| 13.7| 3.2| 8.0| 30.7|33.8*| 22.5|0.13G|999.9|001000| SZ| SWITZERLAND| 0|2019-01-03|
066720|99999|20190104|29.1|20.1|1038.2|982.7| 13.6| 2.4| 4.3|999.9|33.3*| 23.2|0.39G|999.9|000000| SZ| SWITZERLAND| 0|2019-01-04|
066720|99999|20190105|31.6|28.8|1035.2|980.1| 4.4| 1.5| 3.5|999.9|35.1*| 28.8|0.00G|999.9|011000| SZ| SWITZERLAND| 0|2019-01-05|
066720|99999|20190106|36.5|33.4|1033.9|979.4| 4.0| 2.9| 6.4| 12.6|37.8*| 31.5|0.41G|999.9|010000| SZ| SWITZERLAND| 0|2019-01-06|
066720|99999|20190107|36.2|32.1|1034.3|979.8| 8.9| 2.2| 4.7| 9.9|39.6*|33.3*|0.03G| 0.8|010000| SZ| SWITZERLAND| 0|2019-01-07|
066720|99999|20190108|36.7|31.6|1024.1|970.2| 6.0| 7.6| 22.3| 38.5|42.3*|32.0*|0.04G|999.9|010000| SZ| SWITZERLAND| 0|2019-01-08|
066720|99999|20190109|35.1|28.6|1016.6|962.9| 2.9|14.1| 22.0| 42.3|39.2*|31.5*|0.76G| 0.4|011000| SZ| SWITZERLAND| 0|2019-01-09|
066720|99999|20190110|31.8|26.5|1023.2|968.8| 8.5| 3.8| 9.9| 26.4|34.9*| 29.5|0.29G|999.9|001000| SZ| SWITZERLAND| 0|2019-01-10|
066720|99999|20190111|29.9|23.7|1028.2|973.4| 9.0| 1.6| 3.7|999.9|33.1*| 25.3|0.05G|999.9|011000| SZ| SWITZERLAND| 0|2019-01-11|
066720|99999|20190112|32.4|30.1|1027.6|973.0| 6.1| 1.4| 3.5|999.9|35.1*| 29.1|0.05G|999.9|011000| SZ| SWITZERLAND| 0|2019-01-12|
066720|99999|20190113|34.1|33.1|1017.1|963.3| 7.1| 2.8| 7.0| 12.4|36.5*| 31.8|0.65G| 0.8|011000| SZ| SWITZERLAND| 0|2019-01-13|
066720|99999|20190114|42.1|32.7|1014.1|961.3| 3.3|19.4| 24.5| 54.4|47.3*| 34.5|1.50G| 0.4|011000| SZ| SWITZERLAND| 0|2019-01-14|
066720|99999|20190115|36.9|29.4|1024.6|970.7| 12.9| 4.6| 17.1| 41.4|42.8*|28.4*|0.56G|999.9|010000| SZ| SWITZERLAND| 0|2019-01-15|
066720|99999|20190116|31.4|27.4|1018.0|963.9| 13.8| 2.5| 8.4| 16.5|39.4*| 24.8|0.00G|999.9|000000| SZ| SWITZERLAND| 0|2019-01-16|
066720|99999|20190117|37.4|31.8|1013.2|960.0| 10.3| 4.8| 14.2| 28.7|45.9*| 28.2|0.00G|999.9|010000| SZ| SWITZERLAND| 0|2019-01-17|
066720|99999|20190118|31.8|23.1|1019.2|965.1| 10.5| 4.3| 14.0| 26.6|36.5*|25.9*|0.06G|999.9|010000| SZ| SWITZERLAND| 0|2019-01-18|
066720|99999|20190119|27.0|20.4|1017.2|962.6| 12.4| 2.1| 3.5|999.9|33.3*| 21.7|0.00G|999.9|000000| SZ| SWITZERLAND| 0|2019-01-19|
066720|99999|20190120|30.3|22.9|1015.9|961.7| 14.2| 2.2| 6.6| 10.1|38.3*| 23.0|0.00G|999.9|000000| SZ| SWITZERLAND| 0|2019-01-20|
+------+-----+--------+----+----+------+-----+-----+----+-----+-----+-----+-----+-----+-----+------+------------+------------+---+----------+
only showing top 20 rows

In [0]:
# The days with Tornadoes
df = tor.groupby(['COUNTRY_FULL','dt']).agg(F.sum('tor'))
df = df.withColumnRenamed('sum(tor)','tor')
df_order = df.orderBy('COUNTRY_FULL','dt').where(col('tor')>0)
df_order.show()

+------------+----------+---+
COUNTRY_FULL| dt|tor|
+------------+----------+---+
 ALGERIA|2019-07-04|1.0|
 ANGOLA|2019-02-06|1.0|
 ANGOLA|2019-04-05|1.0|
 ANGUILLA|2019-06-06|1.0|
 ARGENTINA|2019-05-10|1.0|
 ARUBA|2019-09-23|1.0|
 AUSTRIA|2019-01-20|1.0|
 AUSTRIA|2019-06-16|1.0|
 AUSTRIA|2019-06-19|1.0|
 AUSTRIA|2019-10-14|1.0|
 AUSTRIA|2019-10-19|1.0|
 BAHAMAS THE|2019-02-25|1.0|
 BAHAMAS THE|2019-03-27|1.0|
 BAHAMAS THE|2019-04-02|1.0|
 BAHAMAS THE|2019-04-15|1.0|
 BAHAMAS THE|2019-04-29|1.0|
 BAHAMAS THE|2019-06-02|1.0|
 BAHAMAS THE|2019-06-15|1.0|
 BAHAMAS THE|2019-07-04|1.0|
 BAHAMAS THE|2019-08-06|1.0|
+------------+----------+---+
only showing top 20 rows

In [0]:
# data with consecutive (*count) days of tornadoes, 
w = Window.partitionBy('COUNTRY_FULL').orderBy('dt')
df_order = df_order.withColumn("rowN", F.row_number().over(w))
df_order = df_order.withColumn("discriminater", col('dt')-col('rowN'))
df_count = df_order.groupBy('COUNTRY_FULL','discriminater').count()
df_count.show()

+------------+-------------+-----+
COUNTRY_FULL|discriminater|count|
+------------+-------------+-----+
 ALGERIA| 2019-07-03| 1|
 ANGOLA| 2019-02-05| 1|
 ANGOLA| 2019-04-03| 1|
 ANGUILLA| 2019-06-05| 1|
 ARGENTINA| 2019-05-09| 1|
 ARUBA| 2019-09-22| 1|
 AUSTRIA| 2019-01-19| 1|
 AUSTRIA| 2019-06-14| 1|
 AUSTRIA| 2019-06-16| 1|
 AUSTRIA| 2019-10-10| 1|
 AUSTRIA| 2019-10-14| 1|
 BAHAMAS THE| 2019-02-24| 1|
 BAHAMAS THE| 2019-03-25| 1|
 BAHAMAS THE| 2019-03-30| 1|
 BAHAMAS THE| 2019-04-11| 1|
 BAHAMAS THE| 2019-04-24| 1|
 BAHAMAS THE| 2019-05-27| 1|
 BAHAMAS THE| 2019-06-08| 1|
 BAHAMAS THE| 2019-06-26| 1|
 BAHAMAS THE| 2019-07-28| 1|
+------------+-------------+-----+
only showing top 20 rows

In [0]:
# the most consecutive days 
df_max = df_count.agg(F.max('count').alias('max'))
df_max.show()

+---+
max|
+---+
 2|
+---+

In [0]:
# The country had the most consecutive days of tornadoes/funnel cloud formations
df_count.join(df_max, df_count['count'] == df_max['max']).select('COUNTRY_FULL').distinct().show()

+--------------+
 COUNTRY_FULL|
+--------------+
 CANADA|
CAYMAN ISLANDS|
 GHANA|
 INDIA|
 ITALY|
 JAPAN|
 UNITED STATES|
+--------------+

In [0]:
# step 2.3 Which country had the second highest average mean wind speed over the year?
data_full_Cname_rmM_wind = data_full_Cname.where(data_full_Cname['WDSP']!=9999.9)
avg_WDSP = data_full_Cname_rmM_wind.groupBy('COUNTRY_FULL').avg('WDSP').select(col('COUNTRY_FULL'),col('avg(WDSP)').alias('avg_WDSP'))
wind_country_2nd = avg_WDSP.orderBy('avg_WDSP',ascending=False).take(2)[-1]['COUNTRY_FULL']
print('The country with the second highest average mean wind speed over the year is: %s' % wind_country_2nd)

The country with the second highest average mean wind speed over the year is: ARMENIA